In [2]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Software Optimizations -- how to write cache friendly code?

## Revisiting the design of your data structures.

### How much space does the following data structures need in physical memory?

In [2]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show="main")])

In [3]:
! cd structure; make clean; make memory_usage_A; ./memory_usage_A

rm -f object_of_arrays memory_usage_A memory_usage_B array_of_objects 
gcc -DHAVE_LINUX_PERF_EVENT_H -g -DA memory_usage.c -o memory_usage_A
40


Now, let's rearrange the data structure a little bit and see what's going on!

In [4]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show=["START_2","END_2"])])
! cd structure; make memory_usage_B; ./memory_usage_B

gcc -DHAVE_LINUX_PERF_EVENT_H -g memory_usage.c -o memory_usage_B
32


### What's a better data structure?

In [5]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START","//END"]),do_render_code("structure/object_of_arrays.c", show=["//START","//END"])])

If the main workload is typically similar to 
```SELECT AVG(assignment_1) FROM table ```

In [6]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START_SELECT","//END_SELECT"]),do_render_code("structure/object_of_arrays.c", show=["//START_SELECT","//END_SELECT"])])

Which one is better?

In [7]:
! cd structure; make array_of_objects; make object_of_arrays

gcc -DHAVE_LINUX_PERF_EVENT_H -g array_of_objects.c -o array_of_objects
gcc -DHAVE_LINUX_PERF_EVENT_H -g object_of_arrays.c -o object_of_arrays


In [8]:
! cd structure; echo "array of objects"; time ./array_of_objects 28800 10000 0; echo "object of arrays"; time ./object_of_arrays 28800 10000 0

array of objects

real	0m4.422s
user	0m3.940s
sys	0m0.360s
object of arrays

real	0m2.690s
user	0m2.250s
sys	0m0.420s


## Loop fusion/fission/interchance

In a very early lecture, we've learned the order of traversing loop matters a lot! It's actually the very first optimization in making your code cache friendly --

### Loop Interchange

In [9]:
compare([do_render_code("madd/madd_A.c",show=["//START","//END"]),do_render_code("madd/madd_B.c",show=["//START","//END"])])

Do you remember the code that let Jetson Nano suffer? What if we change it to the right hand side?

### Loop Fission

In [10]:
compare([do_render_code("loop/loop.c",show=["#else","#endif"]),do_render_code("loop/loop.c",show=["#ifdef A","#else"])])

In [11]:
! ssh htseng@nano "cd courses/CS203/demo/memory/loop; make clean; make; valgrind --tool=cachegrind ./loop_B 524288 >& nano_B.perf;  grep 'D   refs\|D1' nano_B.perf; valgrind --tool=cachegrind ./loop_A 524288 >& nano_A.perf ; grep 'D   refs\|D1' nano_A.perf"

ssh: connect to host nano port 22: No route to host


What if we run it on an intel processor?

In [12]:
! ssh htseng@azelf "cd courses/CS203/demo/memory/loop; make clean; make; valgrind --tool=cachegrind ./loop_B 524288 >& intel_B.perf;  grep 'D   refs\|D1' intel_B.perf; valgrind --tool=cachegrind ./loop_A 524288 >& intel_A.perf ; grep 'D   refs\|D1' intel_A.perf"

rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  loop.c -o loop_B
==322405== D   refs:       75,292,289  (53,783,548 rd   + 21,508,741 wr)
==322405== D1  misses:        724,107  (   330,222 rd   +    393,885 wr)
==322405== D1  miss rate:         1.0% (       0.6%     +        1.8%  )
==322409== D   refs:       74,767,996  (53,521,401 rd   + 21,246,595 wr)
==322409== D1  misses:        658,571  (   264,685 rd   +    393,886 wr)
==322409== D1  miss rate:         0.9% (       0.5%     +        1.9%  )


## Case study: matrix multiplications

GEMM that computes C = A $\times$ B is the core of many AI/ML applications. The most naive implementation of GEMM takes $O(n^3)$. Assume it takes 1 second to perform GEMM on 1,024$\times$1,024$\times$1,024 matrices. How much time do you expect it would take for 2,048$\times$2,048$\times$2,048 matrices?

In [13]:
render_code("matrix_mul/mm.c", show=["//START","//END"])

// matrix_mul/mm.c:71-92 (22 lines)
//START
void mm(double **a, double **b, double **c)
{
  int i,j,k;
  for(i = 0; i < ARRAY_SIZE; i++)
  {
    for(j = 0; j < ARRAY_SIZE; j++)
    {
      for(k = 0; k < ARRAY_SIZE; k++)
      {
        c[i][j] += a[i][k]*b[k][j];
        #ifdef DUMP
          fprintf(stderr, "a[%d][%d], %p\n",i,k, &a[i][k]);
          fprintf(stderr, "b[%d][%d], %p\n",k,j, &b[k][j]);
          fprintf(stderr, "c[%d][%d], %p\n",i,j, &c[i][j]);
        #endif
      }
    }
  }
  return;
}
//END

In [14]:
! cd matrix_mul; make clean; make mm

rm -f blockmm mm blockmm_transpose cachegrind.* mm_dump
gcc -DHAVE_LINUX_PERF_EVENT_H -O3 mm.c perfstats.c -o mm 


In [15]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > mm.csv
! ./matrix_mul/mm 32 >> ./matrix_mul/mm.csv ;./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv
#! cs203 job memory "./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv"

In [16]:
display_df_mono(render_csv("matrix_mul/mm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,314263,105773,0.336575,0.293080,0.000031,0.004962,688,138640
1,9683862988,20837924101,2.151819,0.216177,4.504680,0.315907,1493666440,4728179771
2,77481794355,211460718574,2.729167,0.216384,45.756673,0.353816,13726546060,38795753005


WOW! Compuational complexty breaks again! The GEMM performance go wild because of cache misses!

What kind of misses are we seeing?

In [17]:
! make -C matrix_mul mm_dump; ./matrix_mul/mm_dump 256 >& mm_dump_address.csv

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/matrix_mul'
gcc -DHAVE_LINUX_PERF_EVENT_H -DDUMP -O3 mm.c perfstats.c -o mm_dump 
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/matrix_mul'


In [18]:
! echo "element,address" > mm_dump_addresses_digest.csv 
! head -n 101 mm_dump_address.csv | grep "b\[" >> mm_dump_addresses_digest.csv
df = pd.read_csv("mm_dump_addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 8
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,b[0][0],0x7f68297e1000,0x7f68297e1,0x0
1,b[1][0],0x7f68297e1800,0x7f68297e1,0x20
2,b[2][0],0x7f68297e2000,0x7f68297e2,0x0
3,b[3][0],0x7f68297e2800,0x7f68297e2,0x20
4,b[4][0],0x7f68297e3000,0x7f68297e3,0x0
5,b[5][0],0x7f68297e3800,0x7f68297e3,0x20
6,b[6][0],0x7f68297e4000,0x7f68297e4,0x0
7,b[7][0],0x7f68297e4800,0x7f68297e4,0x20
8,b[8][0],0x7f68297e5000,0x7f68297e5,0x0
9,b[9][0],0x7f68297e5800,0x7f68297e5,0x20


### Matrix tiling algorithm

Let's try to partition GEMM into smaller tiles!

In [19]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

// matrix_mul/blockmm.c:55-73 (19 lines)
//START
void blockmm(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[kk][jj];
      }
    }
  }  
}
//END

In [20]:
! cd matrix_mul/; make blockmm

gcc -O3 -DHAVE_LINUX_PERF_EVENT_H blockmm.c perfstats.c -o blockmm 


In [5]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 32 1 >> ./matrix_mul/blockmm.csv ;./matrix_mul/blockmm 1024 256 >> ./matrix_mul/blockmm.csv ; ./matrix_mul/blockmm 2048 256 >> ./matrix_mul/blockmm.csv

In [6]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,283639,119094,0.419879,0.923640,0.000110,0.005152,725,140726
1,11847820263,3977731807,0.335735,0.215789,0.858351,0.047962,245406056,5116655290
2,79915595610,38566368372,0.482589,0.216032,8.331585,0.066371,2497102945,37623632628


In [23]:
render_code("matrix_mul/blockmm_transpose.c", show=["//START","//END"])

// matrix_mul/blockmm_transpose.c:56-74 (19 lines)
//START
void blockmm_transpose(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[jj][kk];
      }
    }
  }  
}
//END

### Matrix transpose

In [3]:
! cd matrix_mul; make blockmm_transpose; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_transpose.csv
! ./matrix_mul/blockmm_transpose 32 1 >> ./matrix_mul/blockmm_transpose.csv ;./matrix_mul/blockmm_transpose 1024 256 >> ./matrix_mul/blockmm_transpose.csv ; ./matrix_mul/blockmm_transpose 2048 256 >> ./matrix_mul/blockmm_transpose.csv

make: 'blockmm_transpose' is up to date.


In [4]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,249771,124389,0.498012,0.506476,0.000063,0.006869,736,107154
1,10387242744,2402693150,0.231312,0.215816,0.518539,0.018679,75785690,4057159015
2,69989278109,18418683904,0.263164,0.215857,3.975793,0.015937,455029180,28551911266
